In [1]:
import json
from collections import defaultdict
import pandas as pd
from sklearn.model_selection import train_test_split

# Сформировать множество из разных файлов

In [2]:
df = pd.read_csv("../../Data/Raw/merged_with_labels_11_patterns.csv", sep=";")
print(len(df[df['x'].isna()]), "nan texts")
df = df[~df['x'].isna()]

df_3 = pd.read_csv("../../Data/Raw/12052022_one_cmd.csv", sep=";")

df_3.drop("delayed", axis=1, inplace=True)

0 nan texts


In [3]:
df = df

# предобработка

In [ ]:
# в случае изменения функции, изменить её и в обработке длинных команд в Analyze_complex_commands.ipynb
#punct_characters = re.escape(".,!?:;-#$*()[]'\"/|^%~`_+-")
#import re
#def removePunctETC(value):
#    if type(value) is float:
#        return value
#    value = value.lower()
#    value = re.sub("[^\w\s°]", "", value)
#    return value
#df["x"] = df["x"].map(removePunctETC)

# сплит

In [4]:
# сплит сгенерированной части на трейн и валид 
print(df["type"].unique())
traindf = df[~df["type"].isin(['students', 'crowdsource'])]

validIndex = set()
for c in traindf.columns:
    if c in ["x", "y_name", "type"]:
        continue
    train_x, valid_x = train_test_split(traindf.index, test_size=0.03, random_state=69, shuffle=True, stratify=traindf[c])
    validIndex |= set(valid_x)
len(validIndex) / df.shape[0]
subsets = ["valid" if i in validIndex else "train" for i in range(df.shape[0])]
df["subset"] = subsets
df.loc[df["type"].isin(['students', 'crowdsource']),"subset"] = "test"

['new_generator' 'students' 'crowdsource' 'generator']


In [5]:
for c in df.columns:
    if c in ["x", "y_name", "subset", "type"]:
        continue
    trains_counts = df.loc[df["subset"]=="train", c].value_counts().to_dict()
    valid_counts = df.loc[df["subset"]=="valid", c].value_counts().to_dict()
    test_counts = df.loc[df["subset"]=="test", c].value_counts().to_dict()
    print(c)
    for k in trains_counts:
        if k in test_counts:
            print(k, trains_counts[k], valid_counts[k], test_counts[k], valid_counts[k] / trains_counts[k], test_counts[k] / trains_counts[k])
        else:
            print(k, trains_counts[k], valid_counts[k], valid_counts[k] / trains_counts[k])
    print()

y
8 12225 5551 995 0.45406952965235176 0.08139059304703476
7 8391 2905 130 0.3462042664759862 0.015492789893933977
2 4238 762 0.1798017932987258
3 4140 860 0.20772946859903382
4 4095 905 0.221001221001221
11 3664 1336 0.3646288209606987
9 3651 1349 0.36948781155847715
10 3618 1382 0.3819789939192924
6 3097 855 0.27607361963190186
5 2250 518 53 0.23022222222222222 0.023555555555555555
1 555 84 89 0.15135135135135136 0.16036036036036036
0 41 3 51 0.07317073170731707 1.2439024390243902

action
6 29668 11339 1178 0.3821963057840097 0.03970608062558986
5 8235 1765 0.2143290831815422
4 4238 762 0.1798017932987258
0 3668 1350 33 0.3680479825517993 0.008996728462377318
8 1617 578 0.35745207173778604
9 1323 414 0.3129251700680272
7 637 216 0.3390894819466248
2 399 60 56 0.15037593984962405 0.14035087719298245
3 156 24 33 0.15384615384615385 0.21153846153846154
1 24 2 18 0.08333333333333333 0.75

direction
0 36937 13899 1229 0.3762893575547554 0.03327287002192923
3 2354 470 9 0.19966015293118097

In [7]:
df.to_csv("../../Data/Interim/merged_with_labels_11_patterns_split_diff.csv", index=False)
df = pd.read_csv("../../Data/Interim/merged_with_labels_11_patterns_split.csv")
df.sample(2000).to_csv("../../Data/Interim/short_split.csv", index=False)

# folds

In [16]:
from sklearn.model_selection import StratifiedKFold
df = pd.read_csv("../../Data/Interim/merged_with_labels_11_patterns_split.csv")
df["fold"]=-1
df[df["subset"]=="test"]["y"].value_counts()

8    995
7    130
1     89
5     53
0     51
Name: y, dtype: int64

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=42)

for fold_i, (train_index, test_index) in enumerate(skf.split(df[df["subset"]=="test"].index, df[df["subset"]=="test"]["y"])):
    df.loc[df[df["subset"]=="test"].index[test_index], "fold"] = fold_i
    print(len(train_index), len(test_index))
for i in range(5):
    print("fold#", i)
    print(df[df["fold"]==i]["y"].value_counts())

In [ ]:
df.to_csv("../Data/Interim/merged_with_labels_11_patterns_split.csv", index=False)